In [66]:
import torch
import pytorch_lightning as pl
import numpy as np



In [51]:
class FeedForward(torch.nn.Module):

    def __init__(
        self, layers, activation="relu", **kwargs 
    ):
        """
        Define a neural network module given the list of layers.

        Parameters
        ----------
        layers : list
            Number of neurons per layer
        activation : string
            Activation function (relu, tanh, elu, linear)
        random_initialization: bool
            if initialize the weights of the network with random values gaussian distributed N(0,1)

        """
        super().__init__(**kwargs)

        # get activation function
        activ = None
        if activation == "relu":
            activ = torch.nn.ReLU(True)
        elif activation == "elu":
            activ = torch.nn.ELU(True)
        elif activation == "tanh":
            activ = torch.nn.Tanh()
        elif activation == "linear":
            print("WARNING: no activation selected")
        else:
            raise ValueError(
                "Unknown activation. options: 'relu','elu','tanh','linear'. "
            )

        # Create architecture
        if not isinstance(layers[0],int):
            raise TypeError('layers should be a list-type of integers.')
        
        modules = []
        for i in range(len(layers) - 1):
            if i < len(layers) - 2:
                modules.append(torch.nn.Linear(layers[i], layers[i + 1]))
                if activ is not None:
                    modules.append(activ)
            else:
                modules.append(torch.nn.Linear(layers[i], layers[i + 1]))

        self.nn     = torch.nn.Sequential(*modules)
        
        # save models
        self.n_in   = layers[0]
        self.n_out  = layers[-1]

    def forward(self, x: torch.tensor) -> (torch.tensor):
        return self.nn(x)

In [52]:
class Normalization(torch.nn.Module):

    def __init__(self, n_in, criterion='range'):
        
        super().__init__()
        
        self.n_in = n_in
        self.n_out = n_in

        self.criterion = criterion

    def forward(self, x):

        return x
    

class

In [101]:
class SimpleCV(pl.LightningModule):
    
    def __init__(self, modules : list):
        
        super().__init__()
        
        self.blocks = torch.nn.Sequential(*modules)

    def forward(self, x: torch.tensor) -> (torch.tensor):
        y = self.blocks(x)
        return y
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def loss_function(self, input,target):
        mse = torch.nn.MSELoss()
        loss = mse(input,target)
        return loss

    def training_step(self, train_batch, batch_idx):
        x, labels = train_batch
        y = self.blocks(x)
        loss = self.loss_function(y,labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, labels = val_batch
        y = self.blocks(x)
        loss = self.loss_function(y,labels)
        self.log('val_loss', loss)


In [ ]:
n_in = 2 
X = torch.ones(n_in)

layers = [n_in, 10, 1]

params = {  'layers': layers,
            'activation': 'tanh',
            'normIn': True,
            'normOut' : False,
         }

model = SimpleCV( **params )

s = model(X)

Create dataset

In [115]:
from torch.utils.data import TensorDataset, DataLoader, random_split


torch.manual_seed(42)

n_points = 1000

n_in = 2 
n_out = 1

X = torch.randn((n_points,n_in))*10
y = X.square()

dataset = TensorDataset(X,y)
train_dataset, valid_dataset = random_split(dataset, [800,200])

train_loader = DataLoader(train_dataset,batch_size=2)
valid_loader = DataLoader(valid_dataset,batch_size=2) 

Create model and forward

In [118]:

layers = [n_in, 10,10, n_out]

modules = [ Normalization(n_in),
            FeedForward(layers,activation='tanh'),
            Normalization(n_out)]

model = SimpleCV( modules )

print(model)
#print(X)
#model.forward(X)


SimpleCV(
  (blocks): Sequential(
    (0): Normalization()
    (1): FeedForward(
      (nn): Sequential(
        (0): Linear(in_features=2, out_features=10, bias=True)
        (1): Tanh()
        (2): Linear(in_features=10, out_features=10, bias=True)
        (3): Tanh()
        (4): Linear(in_features=10, out_features=1, bias=True)
      )
    )
    (2): Normalization()
  )
)


In [119]:
trainer = pl.Trainer()
trainer.fit(model, train_loader, valid_loader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type       | Params
--------------------------------------
0 | blocks | Sequential | 151   
--------------------------------------
151       Trainable params
0         Non-trainable params
151       Total params
0.001     Total estimated model params size (MB)


Epoch 131:  80%|████████  | 401/500 [00:00<00:00, 433.24it/s, loss=9.14e+03, v_num=54]